In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyomo.environ as pyo
SOLVER = pyo.SolverFactory("ipopt")

ModuleNotFoundError: No module named 'pyomo'

# Household optimization problem

## Problem

For concreteness, suppose that all households are infinitely-lived and identical; that is, each household has the same discount factor $\beta$, the same sequence of effective labor endowments $e(t)^{\infty}_{t=0}$, and the same instantaneous utility function
$u(c^h(t))$,
where $u : R+→ R$ is increasing and concave, and $c^h(t)$ is the consumption of household $h$. Therefore there really is a representative household in this case. Consequently, again ignoring uncertainty, the demand side of the economy can be represented as the solution to the following maximization problem starting at time $t = 0$:

$max \sum ^{\infty}_{t=0} \beta^t u(c(t))$

where $\beta \in (0, 1)$ is the common discount factor of all the households, and $c(t)$ is the consumption level of the representative household.

In [41]:
def household(tfinal, earnings, asset_init, act_period, ret_period):
    m = pyo.ConcreteModel("Household optimization")
    
    m.savRate = pyo.Var(domain=pyo.NonNegativeReals)

    m.actRev = m.savRate * earnings
    m.retAsset = asset_init + act_period * (earnings - m.actRev)
    m.retRev = m.retAsset / ret_period
    

    @m.Objective(sense=pyo.minimize)
    def objective(m):
        return abs(m.retRev - m.actRev)

    @m.Constraint()
    def minActPer(m):
        return m.savRate <= 1.0

    return m

In [42]:
# Specify the initial capital, the risk threshold, and the guaranteed return rate.
tfinal = 10
earnings = 50000
asset_init = 100000
act_period = 20
ret_period = 25

m = household(tfinal, earnings, asset_init, act_period, ret_period)
SOLVER.solve(m)

model.name="Household optimization";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.14.14\x3a Maximum Number of Iterations
      Exceeded.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 1, 'Sense': 'unknown'}], 'Solver': [{'Status': 'warning', 'Message': 'Ipopt 3.14.14\\x3a Maximum Number of Iterations Exceeded.', 'Termination condition': 'maxIterations', 'Id': 400, 'Error rc': 0, 'Time': 0.9433987140655518}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [43]:
def show_table_of_pyomo_variables(X, I, J):
    return pd.DataFrame.from_records(
        [[pyo.value(X[i, j]) for j in J] for i in I], index=I, columns=J
    ).round(2)

print("\nThe corresponding optimal stock levels in each months are:")
display(m.savRate)


The corresponding optimal stock levels in each months are:
savRate : Size=1, Index=None
    Key  : Lower : Value                : Upper : Fixed : Stale : Domain
    None :     0 : 0.009743240110130634 :  None : False : False : NonNegativeReals


In [54]:
print(m.objective())

43123.10839008825
